In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv('diabetes.csv')

In [3]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [4]:
X= df.iloc[:, :-1].values
y= df.iloc[:, -1].values

In [5]:
from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X= ss.fit_transform(X)

In [6]:
X.shape

(768, 8)

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2, random_state=42)

In [8]:
X_train.shape

(614, 8)

In [9]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [10]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss= 'binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.5138 - loss: 0.7503 - val_accuracy: 0.5519 - val_loss: 0.7209
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5333 - loss: 0.7260 - val_accuracy: 0.6169 - val_loss: 0.6759
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6521 - loss: 0.6490 - val_accuracy: 0.6623 - val_loss: 0.6419
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7002 - loss: 0.6131 - val_accuracy: 0.7013 - val_loss: 0.6132
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7068 - loss: 0.5986 - val_accuracy: 0.7143 - val_loss: 0.5911
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7348 - loss: 0.5503 - val_accuracy: 0.7273 - val_loss: 0.5719
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7595 - loss: 0.5176 - val_accuracy: 0.7468 - val_loss: 0.5550
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7516 - loss: 0.5199 - val_accuracy: 0.759

# Making the gridsearch CV equivalent for neural networks


1.   List item
2.   List item



In [12]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [13]:
import kerastuner as kt

<ipython-input-13-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [14]:
 def build_model(hp):


  model= Sequential()

  model.add(Dense(32, activation= 'relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  optimizer= hp.Choice('Optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics= ['accuracy'])

  return model

In [15]:
tuner = kt.RandomSearch(build_model, objective= 'val_accuracy', max_trials=5)

In [16]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 07s]
val_accuracy: 0.7337662577629089

Best val_accuracy So Far: 0.7337662577629089
Total elapsed time: 00h 00m 27s


In [17]:
tuner.get_best_hyperparameters()

In [18]:
tuner.get_best_hyperparameters()[0].values

{'Optimizer': 'sgd'}

In [19]:
model= tuner.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))
#initial epoch means that the next iteration happens from the the 6th one as 5 epochs have already been run

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7146 - loss: 0.5950 - val_accuracy: 0.7143 - val_loss: 0.5871
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7159 - loss: 0.5767 - val_accuracy: 0.7208 - val_loss: 0.5754
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7226 - loss: 0.5653 - val_accuracy: 0.7143 - val_loss: 0.5660
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7217 - loss: 0.5576 - val_accuracy: 0.7273 - val_loss: 0.5582
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7490 - loss: 0.5450 - val_accuracy: 0.7403 - val_loss: 0.5518
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7549 - loss: 0.5384 - val_accuracy: 0.7338 - val_loss: 0.5462
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7437 - loss: 0.5210 - val_accuracy: 0.7273 - val_loss: 0.5416
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7714 - loss: 0.5220 - val_accuracy: 0.74

# now writing a function to find the optimal number of nodes in a layer

In [22]:
def build_model(hp):

  model =Sequential()

  units= hp.Int('units',min_value=8,max_value= 128, step=8) # lower node limit and upper node limit with steps side that will incrase the number of nodes after every iteration

  model.add(Dense(units-units, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss= 'binary_crossentropy', metrics=['accuracy'])

  return model

In [23]:
tuner =kt.RandomSearch(build_model, objective='val_accuracy',
                       max_trials=5,
                       directory='keras_tuning',
                       project_name= 'hyper_parameters')

In [24]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 12s


In [25]:
tuner.get_best_hyperparameters()[0].values

{'units': 88}

In [26]:
model= tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [27]:
model.fit(X_train, y_train, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6513 - loss: 0.6884 - val_accuracy: 0.6429 - val_loss: 0.6865
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6461 - loss: 0.6858 - val_accuracy: 0.6429 - val_loss: 0.6844
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6462 - loss: 0.6836 - val_accuracy: 0.6429 - val_loss: 0.6822
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6907 - loss: 0.6776 - val_accuracy: 0.6429 - val_loss: 0.6805
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6663 - loss: 0.6776 - val_accuracy: 0.6429 - val_loss: 0.6787
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6764 - loss: 0.6744 - val_accuracy: 0.6429 - val_loss: 0.6767
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6727 - loss: 0.6723 - val_accuracy: 0.6429 - val_loss: 0.6751
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6518 - loss: 0.6734 - val_accuracy: 0.64

# to find the optimal number of layers

In [28]:
def build_model(hp):

  model =Sequential()

  model.add(Dense(72, activation='relu', input_dim=8))#initial layer

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

    model.add(Dense(72,activation='relu'))  #checks for number of hidden layers

  model.add(Dense(1, activation='sigmoid')) #final layer

  model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics= ['accuracy'])

  return model


In [29]:
tuner= kt.RandomSearch(build_model, objective ='val_accuracy',
                       max_trials=5, directory= 'keras_tuning',
                       project_name='layers')

In [30]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7597402334213257

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 17s


In [31]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [32]:
model= tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [33]:
model.fit(X_train, y_train, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7650 - loss: 0.4762 - val_accuracy: 0.7662 - val_loss: 0.5196
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7648 - loss: 0.4605 - val_accuracy: 0.7792 - val_loss: 0.5135
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8012 - loss: 0.4202 - val_accuracy: 0.7662 - val_loss: 0.5188
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7738 - loss: 0.4526 - val_accuracy: 0.7597 - val_loss: 0.5297
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8099 - loss: 0.3921 - val_accuracy: 0.7597 - val_loss: 0.5311
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8466 - loss: 0.3810 - val_accuracy: 0.7662 - val_loss: 0.5271
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8204 - loss: 0.3951 - val_accuracy: 0.7597 - val_loss: 0.5382
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8190 - loss: 0.3964 - val_accuracy: 0.75

#Combining all the functions into a single super function

In [34]:
from keras.layers import Dropout

In [35]:
def build_model(hp):
    model = Sequential()

    counter = 0

    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

        if counter == 0:
            # Adding the first Dense layer
            model.add(Dense(hp.Int('units_' + str(i), min_value=8, max_value=128, step=8), activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),input_dim=8))
            model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

        else:
            # Adding subsequent Dense layers
            model.add(Dense(hp.Int('units_' + str(i), min_value=8, max_value=128, step=8), activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])))
            model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

        counter += 1

    # Output layer (for binary classification)
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),  loss='binary_crossentropy', metrics=['accuracy'])

    # Return the built model
    return model


In [36]:
tuner= kt.RandomSearch(build_model, objective ='val_accuracy',
                       max_trials=3, directory= 'keras_tuning',
                       project_name='final_tune')

In [37]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 13s


In [38]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units_0': 16,
 'activation0': 'tanh',
 'dropout0': 0.5,
 'optimizer': 'nadam',
 'units_1': 112,
 'activation1': 'relu',
 'dropout1': 0.5,
 'units_2': 8,
 'activation2': 'sigmoid',
 'dropout2': 0.1,
 'units_3': 72,
 'activation3': 'tanh',
 'dropout3': 0.5,
 'units_4': 120,
 'activation4': 'relu',
 'dropout4': 0.3,
 'units_5': 72,
 'activation5': 'relu',
 'dropout5': 0.8,
 'units_6': 24,
 'activation6': 'tanh',
 'dropout6': 0.8,
 'units_7': 40,
 'activation7': 'relu',
 'dropout7': 0.8}

In [39]:
model= tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 23 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [40]:
model.fit(X_train, y_train, epochs=200, initial_epoch=6, validation_data=(X_test, y_test))


Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5797 - loss: 0.6851 - val_accuracy: 0.6429 - val_loss: 0.6486
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6559 - loss: 0.6618 - val_accuracy: 0.6429 - val_loss: 0.6434
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6833 - loss: 0.6249 - val_accuracy: 0.6429 - val_loss: 0.6348
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6630 - loss: 0.6302 - val_accuracy: 0.6429 - val_loss: 0.6224
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6432 - loss: 0.6256 - val_accuracy: 0.6429 - val_loss: 0.6059
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6978 - loss: 0.5900 - val_accuracy: 0.6494 - val_loss: 0.5825
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6774 - loss: 0.5892 - val_accuracy: 0.7143 - val_loss: 0.5636
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7042 - loss: 0.5656 - val_accuracy: 0.71